# Félix-Antoine Dupuis

<h1 style="text-align:center">Travail pratique numérique en thermodynamique statistique</h1>
<h2 style="text-align:center">PARTIE 1 : Cinétique des gaz parfaits</h2>

Veuillez indiquer le nom des membres de votre équipe dans la cellule suivante.

- Pierre-Olivier Desrosiers<br>
- Nicolas Dorval<br>
- Gérémy Michaud<br>
- Félix-Antoine Dupuis

# Atelier en classe : 31 janvier 2024 #
### Discussion interdisciplinaire de sujets en probablilité & statistiques ###

**Quelques fondements de mathématiques statistiques:** par exemple bien définir variables aléatoires, échantillon et population totale, estimateurs, fonction de distribution cumulative (répartition), densité de probabilité, moments, etc. - **Programme GPH**

**Les distributions statistiques de particules indiscernables:** en particulier celle de Fermi-Dirac avec les notions de potentiel chimique et d’occupation des états en fonction de la température, en analogie avec le remplissage selon le principe principe d’Aufbau introduit en classe pour les atomes à température nulle. - **Programme PHY**

**_Un point de bonus sera accordé à tous les étudiantes et étudiants qui participeront à l'atelier pour discuter des sujets ci-dessus et débuter le travail de la première partie ci-dessous._**

# Introduction #
Ce travail révise d'abord quelques bases générales de mécanique statistique classique avec le script `TDSrevision-2Dsimulation.py` qui simule la théorie cinétique des gaz parfaits en 2D. Cette simulation utilise le concept de sphères dures, mais ici pour le cas des particules d'un gaz afin d'introduire des collisions élastiques entre elles sur leurs trajectoires ballistiques. Notez qu'une sphère est colorée et grossie seulement pour l’effet visuel dans l'animation, la physique de l’algorithme codé considère bien des particules totalement identiques. Les questions sur cette simulation, à répondre directement dans les cellules du carnet _(Notebook)_ ici-même, explorent quelques paramètres de la thermodynamique statistique et introduisent de nouveaux termes utiles à l'étude de la dynamique des électrons dans la matière.

_N.B._ 
- _Pour montrer les animations à l'écran, le script `TDSrevision-2Dsimulation.py` importe la librairie `VPython` qu'il faut donc installer. Des liens vers sa documentation et de l'information complémentaire sont donnés dans la médiagraphie à la fin._
- _Le code dans ce script est abusivement commenté dans notre contexte pédagogique, mais il serait bien sûr préférable de s’en tenir aux recommandations du <a href="https://www.python.org/dev/peps/pep-0008"> PEP 8 — Style Guide for Python Code</a>._
- _Notez finalement que la boucle principale à la fin du script laisse l'utilisateur voir l'animation aussi longtemps que souhaité, assurez-vous donc de savoir comment l'interrompre correctement avant de lancer la simulation ou de la remplacer par une boucle `for`._

# Cinétique CLASSIQUE des gaz parfaits #

### Simulation 2D ###

In [13]:
#!python TDS-2Dsimulation_H24.py
# La simulation peut être exécutée à l'extérieur du _Notebook_ bien sûr, cette cellule ne vise qu'à relier le travail qui suit au programme.

from vpython import *
from IPython.display import display, Math
import numpy as np
import math
import matplotlib.pyplot as plt
import time

In [14]:
start_time = time.time()
# Déclaration de variables influençant le temps d'exécution de la simulation
Natoms = 200  # change this to have more or fewer atoms
dt = 1E-5  # pas d'incrémentation temporel

# Déclaration de variables physiques "Typical values"
mass = 4E-3/6E23 # helium mass
Ratom = 0.01 # wildly exaggerated size of an atom
k = 1.4E-23 # Boltzmann constant
T = 300 # around room temperature

# CANEVAS DE FOND
L = 1 # container is a cube L on a side

# ARÊTES DE BOÎTE 2D
d = L/2+Ratom
r = 0.005

# POSITION ET QUANTITÉ DE MOUVEMENT INITIALE DES SPHÈRES
Atoms = [] # Objet qui contiendra les sphères pour l'animation
p = [] # quantité de mouvement des sphères
apos = [] # position des sphères
pavg = sqrt(2*mass*1.5*k*T) #Principe de l'équipartition de l'énergie en thermodynamique statistique classique

for i in range(Natoms):
    x = L*random()-L/2 # position aléatoire qui tient compte que l'origine est au centre de la boîte
    y = L*random()-L/2
    z = 0
    apos.append(vec(x,y,z)) # liste de la position initiale de toutes les sphères
    phi = 2*pi*random() # direction aléatoire pour la quantité de mouvement
    px = pavg*cos(phi)  # quantité de mvt initiale selon l'équipartition
    py = pavg*sin(phi)
    pz = 0
    p.append(vector(px,py,pz)) # liste de la quantité de mvt initiale de toutes les sphères

# FONCTION POUR IDENTIFIER LES COLLISIONS, I.E. LORSQUE LA DISTANCE ENTRE LES CENTRES DE 2 SPHÈRES EST À LA LIMITE DE S'INTERPÉNÉTRER
def checkCollisions():
    hitlist = []   # initialisation
    r2 = 2*Ratom   # distance critique où les 2 sphères entre en contact à la limite de leur rayon
    r2 *= r2   # produit scalaire pour éviter une comparaison vectorielle ci-dessous
    for i in range(Natoms):
        ai = apos[i]
        for j in range(i) :
            aj = apos[j]
            dr = ai - aj   # la boucle dans une boucle itère pour calculer cette distance vectorielle dr entre chaque paire de sphère
            if mag2(dr) < r2:   # test de collision où mag2(dr) qui retourne la norme élevée au carré de la distance intersphère dr
                hitlist.append([i,j]) # liste numérotant toutes les paires de sphères en collision
    return hitlist

iterations = 500
for i in range(iterations):
    # DÉPLACE TOUTES LES SPHÈRES D'UN PAS SPATIAL deltax
    vitesse = []   # vitesse instantanée de chaque sphère
    deltax = []  # pas de position de chaque sphère correspondant à l'incrément de temps dt
    for i in range(Natoms):
        vitesse.append(p[i]/mass)   # par définition de la quantité de nouvement pour chaque sphère
        deltax.append(vitesse[i] * dt)   # différence avant pour calculer l'incrément de position
        apos[i] = apos[i] + deltax[i]  # nouvelle position de l'atome après l'incrément de temps dt

    # CONSERVE LA QUANTITÉ DE MOUVEMENT AUX COLLISIONS AVEC LES PAROIS DE LA BOÎTE
    for i in range(Natoms):
        loc = apos[i]
        if abs(loc.x) > L/2:
            if loc.x < 0: p[i].x =  abs(p[i].x)  # renverse composante x à la paroi de gauche
            else: p[i].x =  -abs(p[i].x)   # renverse composante x à la paroi de droite
        if abs(loc.y) > L/2:
            if loc.y < 0: p[i].y = abs(p[i].y)  # renverse composante y à la paroi du bas
            else: p[i].y =  -abs(p[i].y)  # renverse composante y à la paroi du haut

    # LET'S FIND THESE COLLISIONS!!!
    hitlist = checkCollisions()

    # CONSERVE LA QUANTITÉ DE MOUVEMENT AUX COLLISIONS ENTRE SPHÈRES
    for ij in hitlist:

        # définition de nouvelles variables pour chaque paire de sphères en collision
        i = ij[0]  # extraction du numéro des 2 sphères impliquées à cette itération
        j = ij[1]
        ptot = p[i]+p[j]   # quantité de mouvement totale des 2 sphères
        mtot = 2*mass    # masse totale des 2 sphères
        Vcom = ptot/mtot   # vitesse du référentiel barycentrique/center-of-momentum (com) frame
        posi = apos[i]   # position de chacune des 2 sphères
        posj = apos[j]
        vi = p[i]/mass   # vitesse de chacune des 2 sphères
        vj = p[j]/mass
        rrel = posi-posj  # vecteur pour la distance entre les centres des 2 sphères
        vrel = vj-vi   # vecteur pour la différence de vitesse entre les 2 sphères

        # exclusion de cas où il n'y a pas de changements à  faire
        if vrel.mag2 == 0: continue  # exactly same velocities si et seulement si le vecteur vrel devient nul, la trajectoire des 2 sphères continue alors côte à côte
        if rrel.mag > Ratom: continue  # one atom went all the way through another, la collision a été "manquée" à l'intérieur du pas deltax

        # calcule la distance et temps d'interpénétration des sphères dures qui ne doit pas se produire dans ce modèle
        dx = dot(rrel, vrel.hat)       # rrel.mag*cos(theta) où theta is the angle between vrel and rrel:
        dy = cross(rrel, vrel.hat).mag # rrel.mag*sin(theta)
        alpha = asin(dy/(2*Ratom))  # alpha is the angle of the triangle composed of rrel, path of atom j, and a line from the center of atom i to the center of atom j where atome j hits atom i
        d = (2*Ratom)*cos(alpha)-dx # distance traveled into the atom from first contact
        deltat = d/vrel.mag         # time spent moving from first contact to position inside atom

        # CHANGE L'INTERPÉNÉTRATION DES SPHÈRES PAR LA CINÉTIQUE DE COLLISION
        posi = posi-vi*deltat   # back up to contact configuration
        posj = posj-vj*deltat
        pcomi = p[i]-mass*Vcom  # transform momenta to center-of-momentum (com) frame
        pcomj = p[j]-mass*Vcom
        rrel = hat(rrel)    # vecteur unitaire aligné avec rrel
        pcomi = pcomi-2*dot(pcomi,rrel)*rrel # bounce in center-of-momentum (com) frame
        pcomj = pcomj-2*dot(pcomj,rrel)*rrel
        p[i] = pcomi+mass*Vcom # transform momenta back to lab frame
        p[j] = pcomj+mass*Vcom
        apos[i] = posi+(p[i]/mass)*deltat # move forward deltat in time, ramenant au même temps où sont rendues les autres sphères dans l'itération
        apos[j] = posj+(p[j]/mass)*deltat
        
print("--- %.3f seconds ---" % (time.time() - start_time))


--- 27.310 seconds ---


### Questions statistiques ###

**I.** _(3 points)_  &mdash; Utilisez la liste finale des vecteurs de quantité de mouvement $\vec{p}$ de toutes les sphères pour trouver la moyenne de son carré $\langle p^2\rangle=\langle\vec{p}\cdot\vec{p}\rangle$ et l'imprimer avec la fonction `print()` dans la cellule qui suit. 

In [15]:
p_squared_avg = sum([mag2(pi) for pi in p]) / Natoms
display(Math(f'\\langle p^2 \\rangle \\approx {p_squared_avg:.3e}'))

<IPython.core.display.Math object>


**II.** _(2 points)_  &mdash; La température $T$ (macroscopique) est proportionnelle à l'énergie cinétique moyenne $E_{cin}$ de l'ensemble des particules lorsque ce système est rendu à l'équilibre. Celle-ci peut se calculer classiquement selon son <a href="https://fr.wikipedia.org/wiki/%C3%89quipartition_de_l%27%C3%A9nergie">principe d'équipartition</a>, _i.e._ répartissant l'énergie également sur chaque degré de liberté ici en translation seulement, d'où au total pour $i=1,2\text{ ou } 3$ dimensions d'espace réel
\begin{equation}
E_{cin}=\frac{\langle p^2 \rangle}{2m}=i\times\frac{1}{2}k_BT
\end{equation}
avec $k_B$, la constante de Boltzmann et $m$, la masse de chaque particule. Quelle est la température du gaz de sphères dures à la fin de la simulation? Est-ce qu'elle a changé significativement par rapport à sa valeur initiale?

Dans notre cas, nous travaillons en 2D ($i = 2$) et la masse $m$ est constante. En réarrangeant l'équation pour résoudre pour la température $T_{final}$, nous obtenons :
\begin{equation*}
T_{final} = \frac{\langle p^2 \rangle}{2mk_B}
\end{equation*}

In [16]:
T_final = p_squared_avg / (2*mass*k)
print(f"Température initiale: {T:.3f} K")
print(f"Température initiale: {T_final:.3f} K")
print(f"La température finale est donc {T_final/T:.2f} fois plus élevée que la température initiale")

Température initiale: 300.000 K
Température initiale: 450.000 K
La température finale est donc 1.50 fois plus élevée que la température initiale


**III.** _(10 points)_ &mdash; Modifiez le code de la simulation pour ajouter une fonction qui suit la trajectoire d'UNE SEULE particule, c'est-à-dire qu'elle doit enregistrer, dans une liste, des valeurs de variables pour cette particule et ce, à chacune de ses collisions avec une autre particule (_i.e._ excluez les collisions avec les parois de la boîte). Les deux variables scalaires à lister sont:
- la distance que la particule a parcouru entre chaque collision,
- le temps écoulé entre ces collisions.

Copiez le code de votre fonction dans la cellule qui suit en y commentant clairement les variables pour ces listes qui devront persister après avoir interrompu l'exécution de la simulation. N'oubliez pas d'inclure votre fichier Python (`.py`) modifié avec la simulation complète lors de la remise.

In [17]:
def suivre_particule(index_particule):
    positions = []  # Liste pour enregistrer les positions de la particule
    temps_entre_collisions = []  # Liste pour enregistrer le temps écoulé entre chaque collision
    distance = []

    for i in range(iterations):
        # Déplace toutes les sphères d'un pas spatial deltax
        vitesse = []   # Vitesse instantanée de chaque sphère
        deltax = []    # Pas de position de chaque sphère correspondant à l'incrément de temps dt

        for j in range(Natoms):
            vitesse.append(p[j]/mass)   # Par définition de la quantité de mouvement pour chaque sphère
            deltax.append(vitesse[j] * dt)   # Différence avant pour calculer l'incrément de position
            apos[j] = apos[j] + deltax[j]  # Nouvelle position de l'atome après l'incrément de temps dt

        # Conserve la quantité de mouvement aux collisions avec les parois de la boîte
        for j in range(Natoms):
            loc = apos[j]
            if abs(loc.x) > L/2:
                if loc.x < 0: p[j].x = abs(p[j].x)  # Renverse composante x à la paroi de gauche
                else: p[j].x = -abs(p[j].x)   # Renverse composante x à la paroi de droite
            if abs(loc.y) > L/2:
                if loc.y < 0: p[j].y = abs(p[j].y)  # Renverse composante y à la paroi du bas
                else: p[j].y = -abs(p[j].y)  # Renverse composante y à la paroi du haut

        # Trouve les collisions
        hitlist = checkCollisions()

        # Conserve la quantité de mouvement aux collisions entre sphères
        for ij in hitlist:
            i = ij[0]
            j = ij[1]
            ptot = p[i] + p[j]
            mtot = 2 * mass
            Vcom = ptot / mtot
            posi = apos[i]
            posj = apos[j]
            vi = p[i] / mass
            vj = p[j] / mass
            rrel = posi - posj
            vrel = vj - vi

            # Exclusion de cas où il n'y a pas de changements à faire
            if vrel.mag2 == 0:
                continue
            if rrel.mag > Ratom:
                continue

            # Calcule la distance et le temps d'interpénétration des sphères
            dx = dot(rrel, vrel.hat)
            dy = cross(rrel, vrel.hat).mag
            alpha = asin(dy / (2 * Ratom))
            d = (2 * Ratom) * cos(alpha) - dx
            deltat = d / vrel.mag
            # Enregistre la position et le temps écoulé pour la particule spécifiée
            if j == index_particule:
                positions.append(posj)
                #distance.append(d[index_particule])
                temps_entre_collisions.append(deltat)

            # Change l'interpénétration des sphères par la cinétique de collision
            posi = posi - vi * deltat
            posj = posj - vj * deltat
            pcomi = p[i] - mass * Vcom
            pcomj = p[j] - mass * Vcom
            rrel = hat(rrel)
            pcomi = pcomi - 2 * dot(pcomi, rrel) * rrel
            pcomj = pcomj - 2 * dot(pcomj, rrel) * rrel
            p[i] = pcomi + mass * Vcom
            p[j] = pcomj + mass * Vcom
            apos[i] = posi + (p[i] / mass) * deltat
            apos[j] = posj + (p[j] / mass) * deltat

    return positions, temps_entre_collisions

# Utilisation de la fonction pour suivre la trajectoire de la particule numéro 0
positions_particule, temps_entre_collisions_particule = suivre_particule(67)

# Affichage des résultat
print(len(positions_particule))
print(sum(temps_entre_collisions_particule))
print("Positions de la particule 0 entre les collisions:", positions_particule)
print("Temps écoulé entre les collisions pour la particule 0:", temps_entre_collisions_particule)

25
0.000302894158729641
Positions de la particule 0 entre les collisions: [<-0.444597, -0.00988372, 0>, <-0.436804, 0.0242017, 0>, <-0.423981, 0.178039, 0>, <-0.30047, 0.306536, 0>, <-0.297496, 0.302156, 0>, <-0.296725, 0.317928, 0>, <-0.476617, 0.420438, 0>, <-0.444441, -0.0621896, 0>, <-0.245468, -0.483506, 0>, <0.0404012, -0.46509, 0>, <0.027927, -0.481638, 0>, <-0.0496587, -0.494629, 0>, <-0.0837736, -0.441998, 0>, <-0.0833759, -0.425316, 0>, <-0.0979457, -0.247689, 0>, <0.469206, -0.437614, 0>, <0.471891, -0.434696, 0>, <0.267987, 0.276961, 0>, <0.142052, 0.419403, 0>, <-0.078848, -0.0697244, 0>, <-0.27903, -0.153895, 0>, <-0.191475, 0.0808101, 0>, <-0.184767, 0.402932, 0>, <-0.301833, 0.387258, 0>, <-0.345104, 0.408924, 0>]
Temps écoulé entre les collisions pour la particule 0: [8.25754150021501e-06, 1.1104061082322552e-05, 1.3618315349626694e-05, 6.6513347879585874e-06, 1.443237327871162e-05, 7.24429745643715e-06, 9.727526802228852e-06, 3.15428178066641e-05, 1.3487302080801057e-

**IV.** _(2 points)_ &mdash; Calculez le **libre parcours moyen** $l_{moy}$ et le **temps de collision** $\tau$ qui sont les valeurs moyennes des deux listes compilées au numéro précédent.

_(Pour votre information, le libre parcours moyen est de l’ordre de 100 nm dans l'air à température et pression ambiantes, mais_ $l_{moy}$ _peut dépasser 100 000 km dans une enceinte sous vide avec les technologies de pompes modernes!)_



In [18]:

#

**V.** _(2 points)_ Calculez la vitesse $\vec{v}$ de la particule entre chaque paire de collisions. Quelle est la vitesse moyenne $\langle\vec{v}\rangle$ de la particule?

In [19]:

#

**VI.** _(5 points)_ &mdash; Pour cette même liste de vitesses, comparez les distributions de la norme $||\vec{v}||$, du carré $v^2$ et d’une de ses composantes $v_x^2$ en étalonnant l’abscisse pour contraster les histogrammes avec une échelle appropriée. Indiquez sur ce graphique la moyenne, le mode, la médiane et la moyenne quadratique des distributions.

In [20]:

#

**Bonus.** _(4 points)_ &mdash; Montrez que 
- (a) le théorème central limite est satisfait par une des distributions de vitesse du numéro précédent,
- (b) le système simulé est ergodique.

In [21]:

#

# Médiagraphie #
 - La simulation utilise la librairie <a href="https://vpython.org">VPython</a> conçue pour faciliter la visualisation de physique en 3D, avec les instructions d’installation <a href="https://vpython.org/presentation2018/install.html">ici</a> et la documentation <a href="https://www.glowscript.org/docs/VPythonDocs/index.html">ici</a>. Le code adapte en 2D et commente en détail l’exemple <a href="https://www.glowscript.org/#/user/GlowScriptDemos/folder/Examples/program/HardSphereGas-VPython">HardSphereGas-VPython</a> du site interactif <a href="https://www.glowscript.org">GlowScript</a> pour programmer des animations avec VPython directement en ligne.